In [10]:
from datasets import load_dataset, Features, Value

fpath = "../data/relevant_incidents_test_minedHN.jsonl"
ds = load_dataset('json', data_files=fpath)['train']

# 새 피처 정의
new_features = Features({
    'id': Value('string'),
    'title': ds.features['title'],
    'query': ds.features['query'],
    'pos': ds.features['pos'],
    'neg': ds.features['neg'],
})

# ds = ds(new_features)


def int_to_str(row):
    row['id'] = str(row['id'])
    return row

ds = ds.map(features=new_features)
# ds = ds.map(int_to_str, features=new_features)
ds[0]

Map:   0%|          | 0/363 [00:00<?, ? examples/s]

{'id': '219',
 'title': '제87조',
 'query': '제87조(내란) 대한민국 영토의 전부 또는 일부에서 국가권력을 배제하거나 국헌을 문란하게 할 목적으로 폭동을 일으킨 자는 다음 각 호의 구분에 따라 처벌한다',
 'pos': ['피의자는 내각 총사퇴와 대통령 퇴진을 명분으로 시민들을 조직, 주요 행정기관을 공격하고 일부 국회의원의 신변을 위협하는 방식으로 국헌을 문란하게 할 것을 기도하였다.'],
 'neg': ['피고인은 회사 내부의 전자기록 문서를 삭제하여 이후 업무 처리가 불가능하도록 함으로써 피해자의 업무 집행을 방해하였다.',
  '피고인은 자신이 소유한 광갱 내에서 폭발물 취급 미숙으로 소규모 폭발 사고를 일으켜 광산 내부 근로자 및 인근 주민에게 공공의 위험을 야기하였다.',
  '피고인은 자기 소유의 상가건물이 타인에게 근저당권 설정된 사실을 알면서도, 근저당권자의 동의 없이 무단으로 임대차계약을 체결하였다.',
  '피고인은 통근 기차가 통과하는 철로에 폭약을 장치하여 기차를 탈선, 전복시켰다.',
  '피고인은 2023년 11월 18일 인터넷 스포츠 도박사이트에 접속해 K리그 경기에 돈을 배팅하여 총 60만원의 도박행위를 하였다.',
  '피의자는 피해자가 부적절한 관계를 가지고 있다는 거짓 주장을 다수에게 전달하여 사회적 평판을 실추시켰다.',
  '피고인은 인터넷 방송 소재를 마련할 목적으로 공동묘지에서 시신 일부를 훼손하고 장난스러운 내용으로 영상을 제작하여 불특정 다수에게 공개하였다.',
  '피고인은 밤 10시경 피해자 E가 점유한 오피스텔 숙소 유리문을 파손한 뒤 방으로 들어가 휴대폰과 현금을 절취하였다.']}

In [11]:
queries = ds.select_columns(['id', 'query'])
queries = queries.rename_column('query', 'text')

corpus = ds.select_columns(['id', 'pos'])
corpus = corpus.rename_column('pos', 'text')

qrels = ds.select_columns(['id'])
qrels = qrels.rename_column('id', 'qid')
qrels = qrels.add_column('docid', ds['id'])
qrels = qrels.add_column('relevance', [1] * len(ds))

In [12]:
queries[1]

{'id': '220', 'text': '우두머리는 사형 무기징역 또는 무기금고에 처한다'}

In [13]:
corpus[1]

{'id': '220', 'text': ['피고인은 내부에서 최고위직을 맡아 내란을 주도하며, 폭동 발생 시 주요 작전을 총괄하였다.']}

In [15]:
qrels[1]

{'qid': '220', 'docid': '220', 'relevance': 1}

In [16]:
queries.to_json("../data/test_queries.jsonl", force_ascii=False)
corpus.to_json("../data/test_corpus.jsonl", force_ascii=False)
qrels.to_json("../data/test_qrels.jsonl", force_ascii=False)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

15246

### KLAID

In [1]:
from datasets import load_dataset
# legal judgment prediction
dataset = load_dataset("lawcompany/KLAID", 'ljp') # Specify a cache directory

README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

KLAID.py:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/60.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/161192 [00:00<?, ? examples/s]

In [ ]:
def only_criminal(row):
    laws = row['laws_service'].split(",")
    if all(law.startswith("형법") for law in laws) : return row

ds_criminal = dataset['train'].filter(only_criminal)

ds_criminal

Filter:   0%|          | 0/161192 [00:00<?, ? examples/s]

Dataset({
    features: ['laws_service_id', 'fact', 'laws_service'],
    num_rows: 89943
})

In [14]:
laws_count = {}

for data in ds_criminal:
  for law in data['laws_service'].split(','):
    if law in laws_count:
      laws_count[law] += 1
    else:
      laws_count[law] = 1

laws_count

{'형법 제260조 제1항': 10224,
 '형법 제314조 제1항': 5927,
 '형법 제332조': 379,
 '형법 제329조': 6442,
 '형법 제257조 제1항': 19406,
 '형법 제245조': 1258,
 '형법 제362조 제1항': 240,
 '형법 제284조': 2438,
 '형법 제283조 제2항': 48,
 '형법 제283조 제1항': 3106,
 '형법 제261조': 1645,
 '형법 제366조': 5136,
 '형법 제319조 제1항': 1933,
 '형법 제347조 제1항': 12278,
 '형법 제355조 제1항': 4221,
 '형법 제323조': 964,
 '형법 제136조 제1항': 11615,
 '형법 제299조': 568,
 '형법 제298조': 2971,
 '형법 제311조': 3771,
 '형법 제258조의2 제1항': 5945,
 '형법 제257조 제2항': 143,
 '형법 제246조 제1항': 430,
 '형법 제144조 제1항': 215,
 '형법 제262조': 549,
 '형법 제259조 제1항': 96,
 '형법 제297조': 297,
 '형법 제319조 제2항': 184,
 '형법 제152조 제1항': 435,
 '형법 제330조': 1326,
 '형법 제356조': 1846,
 '형법 제185조': 531,
 '형법 제307조 제2항': 686,
 '형법 제268조': 504,
 '형법 제355조 제2항': 548,
 '형법 제264조': 112,
 '형법 제331조 제2항': 479,
 '형법 제331조 제1항': 658,
 '형법 제347조 제2항': 105,
 '형법 제156조': 929,
 '형법 제258조 제2항': 27,
 '형법 제258조 제1항': 53,
 '형법 제350조 제1항': 182,
 '형법 제360조 제1항': 363,
 '형법 제307조 제1항': 230,
 '형법 제267조': 26,
 '형법 제347조의2': 193,
 '형법 제369조 제1항': 376,
 '형

In [ ]:
laws_fpath = "../data/queries-edited-filtered.jsonl"
laws_edgecase_fpath = "../data/queries-edited.jsonl"

laws = load_dataset('json', data_files=laws_fpath)['train']
laws_edgecase = load_dataset('json', data_files=laws_edgecase_fpath)['train'].to_pandas()

laws_name = []
for law in laws:
  laws_name.append(law['title'])

laws_set = set(laws_name)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
print("<<누락>>\n")

for law in laws_count.keys():
  if law not in laws_set:
    print(law)
    print(laws_edgecase.loc[laws_edgecase['title'] == law]['content'])

<<누락>>

형법 제332조
688    제332조(상습범) 상습으로 제329조 내지 제331조의2의 죄를 범한 자는 그 죄...
Name: content, dtype: object
형법 제264조
545    제264조(상습범) 상습으로 제257조 제258조 제258조의2 제260조 또는 제...
Name: content, dtype: object
형법 제285조
591    제285조(상습범) 상습으로 제283조제1항 제2항 또는 전조의 죄를 범한 때에는 ...
Name: content, dtype: object


In [15]:
ds_criminal[0]

{'laws_service_id': 34,
 'fact': '피고인 A은 노동일에 종사 중이다. 피고인은 2017. 2. 2. 20:00경 부산 해운대구 B, C식당 앞 노상에서 술을 마시고 D의 포터차량을 타려다가 주변에 있던 E 일행이 음주운전을 하려 한다고 말을 건 일로 서로 시비가 되었다. 피고인은 피해자 E과 시비를 벌이다. 피해자를 밀치고, 인근 주차장에서 들고 온 약 1.5m 길이의 쇠막대기로 E의 머리와 어깨 부위를 4~5회 가량 내리쳐 피해자를 폭행하였다.',
 'laws_service': '형법 제260조 제1항'}

In [95]:
laws[0]

{'id': 0, 'title': '형법 제1조', 'content': '제1조(범죄의 성립과 처벌)'}

In [53]:
from datasets import Dataset, Features, Value
import json


def mapper(
    keyword,
    laws: Dataset,
):
    df_laws = laws.to_pandas()
    # 검색
    matched = df_laws.loc[df_laws["title"] == keyword]

    if not matched.empty:
        id = str(matched.index[0])
        text = matched["content"].iloc[0]
        return id, text
    else:
        raise ValueError(f"No results were found for the keyword {keyword}")


def trans_to_testset(dataset: Dataset, laws: Dataset, fname_prefix: str = ""):
    queries = laws.cast_column('id', Value('string'))
    queries = queries.rename_column('content', 'text')
    queries = queries.remove_columns(['title'])

    qids = []
    corpus = []
    qrels = []
    for i, data in enumerate(dataset):
        docid = str(i)
        corpus.append({"id": docid, "text": [data["fact"]]})
        for keyword in list(map(lambda x: x.strip(), data["laws_service"].split(","))):
            qid, _ = mapper(keyword, laws)
            qids.append(qid)
            qrels.append({"qid": qid, "docid": docid, "relevance": 1})

    queries = queries.filter(lambda x : x if x['id'] in qids else None)

    # save
    fname_prefix = f"{fname_prefix}_" if fname_prefix != "" else ""
    queries.to_json(fname_prefix + "test_queries.jsonl", force_ascii=False)

    with open(fname_prefix + "test_corpus.jsonl", "w", encoding="utf-8") as f:
        for c in corpus:
            json.dump(c, f, ensure_ascii=False)
            f.write("\n")

    with open(fname_prefix + "test_qrels.jsonl", "w", encoding="utf-8") as f:
        for q in qrels:
            json.dump(q, f, ensure_ascii=False)
            f.write("\n")

In [54]:
from datasets import load_dataset
# legal judgment prediction
dataset = load_dataset("lawcompany/KLAID", 'ljp') # Specify a cache directory

def only_criminal(row):
    laws = row['laws_service'].split(",")
    if all(law.startswith("형법") for law in laws) : return row

ds_criminal = dataset['train'].filter(only_criminal)

laws = load_dataset('json', data_files="../data/queries-edited.jsonl")['train']

In [55]:
trans_to_testset(ds_criminal.select([i for i in range(0, 3000)]), laws, "KLAID")

Filter:   0%|          | 0/762 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [50]:
from datasets import load_dataset

dataset_dir = "../data"

queries = load_dataset("json", data_files=f"{dataset_dir}/KLAID_test_queries.jsonl")["train"]
corpus = load_dataset("json", data_files=f"{dataset_dir}/KLAID_test_corpus.jsonl")["train"]
qrels = load_dataset("json", data_files=f"{dataset_dir}/KLAID_test_qrels.jsonl")["train"]

queries_text = queries["text"]
corpus_text = [text for sub in corpus["text"] for text in sub]

qrels_dict = {}
for line in qrels:
    if line["qid"] not in qrels_dict:
        qrels_dict[line["qid"]] = {}
    qrels_dict[line["qid"]][line["docid"]] = line["relevance"]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

### evaluation.py

In [ ]:
from datasets import load_dataset


## data loading ##

dataset_dir = "/data2/local_datasets/bge-m3/data"
# queries = load_dataset("json", data_files=f"{dataset_dir}/test_queries.jsonl")["train"]
# corpus = load_dataset("json", data_files=f"{dataset_dir}/test_corpus.jsonl")["train"]
# qrels = load_dataset("json", data_files=f"{dataset_dir}/test_qrels.jsonl")["train"]

queries = load_dataset("json", data_files=f"{dataset_dir}/KLAID_test_queries.jsonl")["train"]
corpus = load_dataset("json", data_files=f"{dataset_dir}/KLAID_test_corpus.jsonl")["train"]
qrels = load_dataset("json", data_files=f"{dataset_dir}/KLAID_test_qrels.jsonl")["train"]

queries_text = queries["text"]
corpus_text = [text for sub in corpus["text"] for text in sub]

qrels_dict = {}
for line in qrels:
    if line["qid"] not in qrels_dict:
        qrels_dict[line["qid"]] = {}
    qrels_dict[line["qid"]][line["docid"]] = line["relevance"]


## Similarity search method ##

import faiss
import numpy as np
from tqdm import tqdm
from typing import Dict, List, Tuple
from collections import defaultdict


def search(model, queries_text, corpus_text):

    queries_embeddings = model.encode_queries(queries_text)
    corpus_embeddings = model.encode_corpus(corpus_text)

    # create and store the embeddings in a Faiss index
    dim = corpus_embeddings.shape[-1]
    index = faiss.index_factory(dim, "Flat", faiss.METRIC_INNER_PRODUCT)
    corpus_embeddings = corpus_embeddings.astype(np.float32)
    index.train(corpus_embeddings)
    index.add(corpus_embeddings)

    query_size = len(queries_embeddings)

    all_scores = []
    all_indices = []

    # search top 100 answers for all the queries
    for i in tqdm(range(0, query_size, 32), desc="Searching"):
        j = min(i + 32, query_size)
        query_embedding = queries_embeddings[i:j]
        score, indice = index.search(query_embedding.astype(np.float32), k=100)
        all_scores.append(score)
        all_indices.append(indice)

    all_scores = np.concatenate(all_scores, axis=0)
    all_indices = np.concatenate(all_indices, axis=0)

    # store the results into the format for evaluation
    results = {}
    for idx, (scores, indices) in enumerate(zip(all_scores, all_indices)):
        results[queries["id"][idx]] = {}
        for score, index in zip(scores, indices):
            if index != -1:
                results[queries["id"][idx]][corpus["id"][index]] = float(score)

    return results

## debugging
def evaluate_mrr(
    qrels: Dict[str, Dict[str, int]],
    results: Dict[str, Dict[str, float]],
    k_values: List[int],
) -> Tuple[Dict[str, float]]:
    mrr = defaultdict(list)

    k_max, top_hits = max(k_values), {}

    # debugging
    qids = []
    for query_id, doc_scores in results.items():
        # debugging
        qids.append(query_id)
        top_hits[query_id] = sorted(
            doc_scores.items(), key=lambda item: item[1], reverse=True
        )[0:k_max]

    print(qids)

    for query_id in top_hits:
        query_relevant_docs = {
            doc_id for doc_id in qrels[query_id] if qrels[query_id][doc_id] > 0
        }
        for k in k_values:
            rr = 0
            for rank, hit in enumerate(top_hits[query_id][0:k], 1):
                if hit[0] in query_relevant_docs:
                    rr = 1.0 / rank
                    break
            mrr[f"MRR@{k}"].append(rr)

    for k in k_values:
        mrr[f"MRR@{k}"] = round(sum(mrr[f"MRR@{k}"]) / len(qrels), 5)

    return mrr


## Evaluation ##

from FlagEmbedding.abc.evaluation.utils import evaluate_metrics, evaluate_mrr
from FlagEmbedding import FlagModel

k_values = [10, 100]

raw_name = "BAAI/bge-m3"
finetuned_path = "/data2/local_datasets/bge-m3/ft_model"

## Raw model

raw_model = FlagModel(
    raw_name,
    # query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
    devices=[0],
    use_fp16=False,
)

results = search(raw_model, queries_text, corpus_text)

eval_res = evaluate_metrics(qrels_dict, results, k_values)
mrr = evaluate_mrr(qrels_dict, results, k_values)

print("## Raw model ##")

for res in eval_res:
    print(res)
print(mrr)

## Fine tuning model

ft_model = FlagModel(
    finetuned_path,
    # query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
    devices=[0],
    use_fp16=False,
)

results = search(ft_model, queries_text, corpus_text)

eval_res = evaluate_metrics(qrels_dict, results, k_values)
mrr = evaluate_mrr(qrels_dict, results, k_values)

print("## Fine-tuning model ##")

for res in eval_res:
    print(res)
print(mrr)


### evaluation.sh

In [ ]:
#!/usr/bin/bash
#SBATCH -J bge-m3
#SBATCH --gres=gpu:1
#SBATCH --cpus-per-gpu=8
#SBATCH --mem-per-gpu=32G
#SBATCH -w aurora-g6
#SBATCH -p batch_ugrad
#SBATCH -t 1-0
#SBATCH -o logs/slurm-%A.out

python evaluation.py